In [7]:
import mysql.connector

# Direct connection
conn = mysql.connector.connect(
    host="127.0.0.1", 
    user="root",
    password="sql123",
    database="cricbuzz_livestats",
    port=3306
)

cursor = conn.cursor()

In [3]:
import requests

url = "https://cricbuzz-cricket.p.rapidapi.com/matches/v1/recent"

headers = {
	"x-rapidapi-key": "ac9352cb48msh05322e9d78564bfp133fa0jsn499d30d09b05",
	"x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
}

response = requests.get(url, headers=headers)

data = response.json()

In [5]:
data

{'typeMatches': [{'matchType': 'International',
   'seriesMatches': [{'seriesAdWrapper': {'seriesId': 8788,
      'seriesName': 'South Africa tour of England, 2025',
      'matches': [{'matchInfo': {'matchId': 105794,
         'seriesId': 8788,
         'seriesName': 'South Africa tour of England, 2025',
         'matchDesc': '3rd ODI',
         'matchFormat': 'ODI',
         'startDate': '1757239200000',
         'endDate': '1757268000000',
         'state': 'Complete',
         'status': 'England won by 342 runs',
         'team1': {'teamId': 9,
          'teamName': 'England',
          'teamSName': 'ENG',
          'imageId': 172123},
         'team2': {'teamId': 11,
          'teamName': 'South Africa',
          'teamSName': 'RSA',
          'imageId': 172126},
         'venueInfo': {'id': 21,
          'ground': 'The Rose Bowl',
          'city': 'Southampton',
          'timezone': '+01:00',
          'latitude': '50.924131',
          'longitude': '-1.322163'},
         'currB

In [8]:
import datetime
import re

for type_match in data['typeMatches']:
    for series in type_match['seriesMatches']:
        if "seriesAdWrapper" not in series:
            continue  

        series_data = series['seriesAdWrapper']

        for match in series_data['matches']:
            info = match['matchInfo']

            series_name = info.get('seriesName')
            match_desc = info.get('matchDesc')
            teams = info['team1']['teamName'] + " vs " + info['team2']['teamName']
            venue = info['venueInfo']['ground']
            city = info['venueInfo']['city']
            timestamp_ms = int(info.get('startDate'))
            match_date = datetime.datetime.fromtimestamp(timestamp_ms / 1000)

            # --- Winner from stateTitle ---
            state_title = info.get("stateTitle", "")
            winner = None
            victory_margin, victory_type = None, None

            if "Won" in state_title:
                winner = state_title.replace(" Won", "")
            elif "Tied" in state_title:
                winner = "Match Tied"
            elif "Abandoned" in state_title:
                winner = "Match Abandoned"

            # --- Victory margin/type from status ---
            status = info.get("status", "")
            m = re.match(r".+ won by (\d+) (\w+)", status)
            if m:
                victory_margin = int(m.group(1))
                victory_type = m.group(2)

            print("Series:", series_name)
            print("Match:", match_desc)
            print("Teams:", teams)
            print("Venue:", venue, "-", city)
            print("Match Date:", match_date)
            print("Winner:", winner)
            print("Margin:", victory_margin)
            print("Type:", victory_type)
            print("-" * 50)

            # Insert into DB
            cursor.execute("""
                INSERT INTO recent_matches10
                (series_name, match_desc, teams, venue, city, match_date, winner, victory_margin, victory_type)
                VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)
            """, (
                series_name,
                match_desc,
                teams,
                venue,
                city,
                match_date,
                winner,
                victory_margin,
                victory_type
            ))
            conn.commit()


Series: South Africa tour of England, 2025
Match: 3rd ODI
Teams: England vs South Africa
Venue: The Rose Bowl - Southampton
Match Date: 2025-09-07 15:30:00
Winner: ENG
Margin: 342
Type: runs
--------------------------------------------------
Series: South Africa tour of England, 2025
Match: 1st ODI
Teams: England vs South Africa
Venue: Headingley - Leeds
Match Date: 2025-09-02 17:30:00
Winner: RSA
Margin: 7
Type: wkts
--------------------------------------------------
Series: South Africa tour of England, 2025
Match: 2nd ODI
Teams: South Africa vs England
Venue: Lord's - London
Match Date: 2025-09-04 17:30:00
Winner: RSA
Margin: 5
Type: runs
--------------------------------------------------
Series: United Arab Emirates T20I Tri-Series 2025
Match: Final
Teams: Pakistan vs Afghanistan
Venue: Sharjah Cricket Stadium - Sharjah
Match Date: 2025-09-07 20:30:00
Winner: PAK
Margin: 75
Type: runs
--------------------------------------------------
Series: United Arab Emirates T20I Tri-Series 20

In [14]:
import datetime
for type_match in data['typeMatches']:
    # print("Match Type:", type_match['matchType'])

    for series in type_match['seriesMatches']:
        # Some entries are ads, skip them
        if "seriesAdWrapper" not in series:
            continue  

        series_data = series['seriesAdWrapper']

        for match in series_data['matches']:
            info = match['matchInfo']
            score = match.get('matchScore', {})  # safely get matchScore

            print("Series:", info.get('seriesName'))
            print("Match:", info.get('matchDesc'))
            print("Team 1:", info['team1']['teamName'])
            print("Team 2:", info['team2']['teamName'])
            teams = info['team1']['teamName'] + " vs " + info['team2']['teamName']
            print("Venue:", info['venueInfo']['ground'], "-", info['venueInfo']['city'])
            timestamp_ms = int(info.get('startDate'))
            date = datetime.datetime.fromtimestamp(timestamp_ms / 1000)
            print("Match Date: ",date)

            cursor.execute("insert into recent_matches values (%s,%s,%s,%s,%s,%s)",
                        (info.get('seriesName'), info.get('matchDesc'), teams, info['venueInfo']['ground'], info['venueInfo']['city'], date))

            conn.commit()

Series: South Africa tour of England, 2025
Match: 3rd ODI
Team 1: England
Team 2: South Africa
Venue: The Rose Bowl - Southampton
Match Date:  2025-09-07 15:30:00
Series: South Africa tour of England, 2025
Match: 1st ODI
Team 1: England
Team 2: South Africa
Venue: Headingley - Leeds
Match Date:  2025-09-02 17:30:00
Series: South Africa tour of England, 2025
Match: 2nd ODI
Team 1: South Africa
Team 2: England
Venue: Lord's - London
Match Date:  2025-09-04 17:30:00
Series: United Arab Emirates T20I Tri-Series 2025
Match: Final
Team 1: Pakistan
Team 2: Afghanistan
Venue: Sharjah Cricket Stadium - Sharjah
Match Date:  2025-09-07 20:30:00
Series: United Arab Emirates T20I Tri-Series 2025
Match: 3rd Match
Team 1: Afghanistan
Team 2: United Arab Emirates
Venue: Sharjah Cricket Stadium - Sharjah
Match Date:  2025-09-01 20:30:00
Series: United Arab Emirates T20I Tri-Series 2025
Match: 5th Match
Team 1: Pakistan
Team 2: United Arab Emirates
Venue: Sharjah Cricket Stadium - Sharjah
Match Date:  2

In [17]:
#Venue
import requests
data = []
for i in range(45,70):
    url = f"https://cricbuzz-cricket.p.rapidapi.com/venues/v1/{i}"

    headers = {
        "x-rapidapi-key": "fe9c1d0ff9mshf8e519597a3627fp1b6d45jsn79f69a8379bc",
        "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers)
    try:
        data.append(response.json())
    except Exception:
        print(i)

In [31]:
data

[{'ground': 'Basin Reserve',
  'city': 'Wellington',
  'country': 'New Zealand',
  'timezone': '+13:00',
  'capacity': '11,600',
  'ends': 'Vance Stand End, Scoreboard End',
  'homeTeam': 'Wellington',
  'imageUrl': 'http://i.cricketcb.com/i/stats/fth/540x303/venue/images/45.jpg',
  'imageId': '189174'},
 {'ground': 'Punjab Cricket Association IS Bindra Stadium',
  'city': 'Mohali',
  'country': 'India',
  'timezone': '+05:30',
  'established': 1993,
  'capacity': '30000',
  'ends': 'Pavilion End, City End',
  'homeTeam': 'Punjab, Punjab Kings',
  'floodlights': True,
  'curator': 'Daljit Singh',
  'profile': '<b>Venue description:</b>\n\nBack in 1992, builders in Mohali transformed what was basically a swampland into one of the finest stadiums in the country which came to be known as The Punjab Cricket Association Stadium. It is located on the outskirts of Chandigarh. It is a world-class stadium, known for high-quality practice wickets and good spectator facilities. However, there hav

In [44]:

for ven in data[1:]:
    venue = ven.get('ground')
    city = ven.get('city')
    country = ven.get('country')
    capacity = ven.get('capacity')
    # capacity = int(str(capacity).replace(",", ""))

    cursor.execute("insert into venue values(%s,%s,%s,%s)",(venue,city,country,capacity))
    conn.commit()
        


In [ ]:
import requests

url = "https://cricbuzz-cricket.p.rapidapi.com/stats/v1/topstats"

headers = {
	"x-rapidapi-key": "fe9c1d0ff9mshf8e519597a3627fp1b6d45jsn79f69a8379bc",
	"x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
}

response = requests.get(url, headers=headers)

data = response.json()

In [6]:
data

{'statsTypesList': [{'types': [{'value': 'mostRuns',
     'header': 'Most Runs',
     'category': 'Batting'},
    {'value': 'highestScore',
     'header': 'Highest Scores',
     'category': 'Batting'},
    {'value': 'highestAvg',
     'header': 'Best Batting Average',
     'category': 'Batting'},
    {'value': 'highestSr',
     'header': 'Best Batting Strike Rate',
     'category': 'Batting'},
    {'value': 'mostHundreds',
     'header': 'Most Hundreds',
     'category': 'Batting'},
    {'value': 'mostFifties', 'header': 'Most Fifties', 'category': 'Batting'},
    {'value': 'mostFours', 'header': 'Most Fours', 'category': 'Batting'},
    {'value': 'mostSixes', 'header': 'Most Sixes', 'category': 'Batting'},
    {'value': 'mostNineties',
     'header': 'Most Nineties',
     'category': 'Batting'}],
   'category': 'Batting'},
  {'types': [{'value': 'mostWickets',
     'header': 'Most Wickets',
     'category': 'Bowling'},
    {'value': 'lowestAvg',
     'header': 'Best Bowling Average',


In [7]:
import requests

url = "https://cricbuzz-cricket.p.rapidapi.com/stats/v1/team/2"

headers = {
	"x-rapidapi-key": "fe9c1d0ff9mshf8e519597a3627fp1b6d45jsn79f69a8379bc",
	"x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
}

response = requests.get(url, headers=headers)

data = response.json()

In [8]:
data

{'types': [{'header': 'Batting'},
  {'value': 'mostRuns', 'header': 'Most Runs', 'category': 'Batting'},
  {'value': 'highestScore', 'header': 'Highest Scores', 'category': 'Batting'},
  {'value': 'highestAvg',
   'header': 'Best Batting Average',
   'category': 'Batting'},
  {'value': 'highestSr',
   'header': 'Best Batting Strike Rate',
   'category': 'Batting'},
  {'value': 'mostHundreds', 'header': 'Most Hundreds', 'category': 'Batting'},
  {'value': 'mostFifties', 'header': 'Most Fifties', 'category': 'Batting'},
  {'value': 'mostFours', 'header': 'Most Fours', 'category': 'Batting'},
  {'value': 'mostSixes', 'header': 'Most Sixes', 'category': 'Batting'},
  {'value': 'mostNineties', 'header': 'Most Nineties', 'category': 'Batting'},
  {'header': 'Bowling'},
  {'value': 'mostWickets', 'header': 'Most Wickets', 'category': 'Bowling'},
  {'value': 'lowestAvg',
   'header': 'Best Bowling Average',
   'category': 'Bowling'},
  {'value': 'bestBowlingInnings',
   'header': 'Best Bowling

In [4]:
import requests

url = "https://cricbuzz-cricket.p.rapidapi.com/stats/v1/topstats/0"

querystring = {"statsType":"highestScore","matchType":"3"}

headers = {
	"x-rapidapi-key": "fe9c1d0ff9mshf8e519597a3627fp1b6d45jsn79f69a8379bc",
	"x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

data = response.json()
data

{'filter': {'matchtype': [{'matchTypeId': '1', 'matchTypeDesc': 'test'},
   {'matchTypeId': '2', 'matchTypeDesc': 'odi'},
   {'matchTypeId': '3', 'matchTypeDesc': 't20'}],
  'team': [{'id': '27', 'teamShortName': 'IRE'},
   {'id': '10', 'teamShortName': 'WI'},
   {'id': '9', 'teamShortName': 'ENG'},
   {'id': '6', 'teamShortName': 'BAN'},
   {'id': '5', 'teamShortName': 'SL'},
   {'id': '4', 'teamShortName': 'AUS'},
   {'id': '3', 'teamShortName': 'PAK'},
   {'id': '11', 'teamShortName': 'RSA'},
   {'id': '12', 'teamShortName': 'ZIM'},
   {'id': '13', 'teamShortName': 'NZ'},
   {'id': '2', 'teamShortName': 'IND'},
   {'id': '96', 'teamShortName': 'AFG'}],
  'selectedMatchType': 't20'},
 'headers': ['Batter', 'HS', 'Balls', 'SR', 'Vs'],
 'values': [{'values': ['1643', 'Finch', '172', '76', '226.32', 'ZIM']},
  {'values': ['41897', 'Lucia Taylor', '169', '84', '201.19', 'CHLW']},
  {'values': ['21193', 'Zeeshan Khan', '164', '84', '195.24', 'HUN']},
  {'values': ['12068', 'Hazratullah Za

In [5]:
import requests

url = "https://cricbuzz-cricket.p.rapidapi.com/series/v1/archives/international"

querystring = {"year":"2024"}

headers = {
	"x-rapidapi-key": "fe9c1d0ff9mshf8e519597a3627fp1b6d45jsn79f69a8379bc",
	"x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

print(response.json())

{'seriesMapProto': [{'date': '2024', 'series': [{'id': 8553, 'name': 'Sri Lanka tour of New Zealand, 2024-25', 'startDt': '1734912000000', 'endDt': '1736553600000'}, {'id': 9297, 'name': 'Gulf Cricket T20I Championship, 2024', 'startDt': '1734048000000', 'endDt': '1734739200000'}, {'id': 8044, 'name': 'Pakistan tour of South Africa, 2024 -25', 'startDt': '1733788800000', 'endDt': '1736208000000'}, {'id': 9171, 'name': 'Afghanistan tour of Zimbabwe, 2024-25', 'startDt': '1733702400000', 'endDt': '1736121600000'}, {'id': 9265, 'name': 'ICC Mens T20 World Cup Americas Sub Regional Qualifier, 2024', 'startDt': '1733443200000', 'endDt': '1734307200000'}, {'id': 9289, 'name': 'Africa Continental Cup 2024', 'startDt': '1733270400000', 'endDt': '1734134400000'}, {'id': 8058, 'name': 'Sri Lanka tour of South Africa, 2024', 'startDt': '1732665600000', 'endDt': '1733702400000'}, {'id': 8494, 'name': 'Pakistan tour of Zimbabwe, 2024', 'startDt': '1732406400000', 'endDt': '1733356800000', 'isFantas

In [6]:
data = response.json()
data

{'seriesMapProto': [{'date': '2024',
   'series': [{'id': 8553,
     'name': 'Sri Lanka tour of New Zealand, 2024-25',
     'startDt': '1734912000000',
     'endDt': '1736553600000'},
    {'id': 9297,
     'name': 'Gulf Cricket T20I Championship, 2024',
     'startDt': '1734048000000',
     'endDt': '1734739200000'},
    {'id': 8044,
     'name': 'Pakistan tour of South Africa, 2024 -25',
     'startDt': '1733788800000',
     'endDt': '1736208000000'},
    {'id': 9171,
     'name': 'Afghanistan tour of Zimbabwe, 2024-25',
     'startDt': '1733702400000',
     'endDt': '1736121600000'},
    {'id': 9265,
     'name': 'ICC Mens T20 World Cup Americas Sub Regional Qualifier, 2024',
     'startDt': '1733443200000',
     'endDt': '1734307200000'},
    {'id': 9289,
     'name': 'Africa Continental Cup 2024',
     'startDt': '1733270400000',
     'endDt': '1734134400000'},
    {'id': 8058,
     'name': 'Sri Lanka tour of South Africa, 2024',
     'startDt': '1732665600000',
     'endDt': '1733

In [24]:
from datetime import datetime
d = data['seriesMapProto'][0]['series']
d
for d1 in d:
    s_id = d1["id"]
    name = d1["name"]
    start_date = datetime.utcfromtimestamp(int(d1["startDt"]) / 1000).strftime("%Y-%m-%d")
    end_date = datetime.utcfromtimestamp(int(d1["endDt"]) / 1000).strftime("%Y-%m-%d")
    # print(f"ID: {s_id} | Name: {name} | Start: {start_date} | End: {end_date}")
    cursor.execute(""" insert into series_2024 (series_name, start_date, id) values (%s,%s,%s)
                   """,
                   (name,start_date,s_id))
    conn.commit()

C:\Users\hameedbf\AppData\Local\Temp\ipykernel_4208\2333295568.py:7: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  start_date = datetime.utcfromtimestamp(int(d1["startDt"]) / 1000).strftime("%Y-%m-%d")
C:\Users\hameedbf\AppData\Local\Temp\ipykernel_4208\2333295568.py:8: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  end_date = datetime.utcfromtimestamp(int(d1["endDt"]) / 1000).strftime("%Y-%m-%d")


In [46]:
import requests

url = "https://cricbuzz-cricket.p.rapidapi.com/stats/v1/series/8553/points-table"

headers = {
	"x-rapidapi-key": "fe9c1d0ff9mshf8e519597a3627fp1b6d45jsn79f69a8379bc",
	"x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
}

response = requests.get(url, headers=headers)
response.status_code
# data = response.json()

204

In [42]:
data

{'types': [{'header': 'Batting'},
  {'value': 'mostRuns', 'header': 'Most Runs', 'category': 'Batting'},
  {'value': 'highestScore', 'header': 'Highest Scores', 'category': 'Batting'},
  {'value': 'highestAvg',
   'header': 'Best Batting Average',
   'category': 'Batting'},
  {'value': 'highestSr',
   'header': 'Best Batting Strike Rate',
   'category': 'Batting'},
  {'value': 'mostHundreds', 'header': 'Most Hundreds', 'category': 'Batting'},
  {'value': 'mostFifties', 'header': 'Most Fifties', 'category': 'Batting'},
  {'value': 'mostFours', 'header': 'Most Fours', 'category': 'Batting'},
  {'value': 'mostSixes', 'header': 'Most Sixes', 'category': 'Batting'},
  {'value': 'mostNineties', 'header': 'Most Nineties', 'category': 'Batting'},
  {'header': 'Bowling'},
  {'value': 'mostWickets', 'header': 'Most Wickets', 'category': 'Bowling'},
  {'value': 'lowestAvg',
   'header': 'Best Bowling Average',
   'category': 'Bowling'},
  {'value': 'bestBowlingInnings',
   'header': 'Best Bowling

In [51]:
url = "https://cricbuzz-cricket.p.rapidapi.com/stats/v1/player/587/batting"

headers = {
	"x-rapidapi-key": "fe9c1d0ff9mshf8e519597a3627fp1b6d45jsn79f69a8379bc",
	"x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
}

response = requests.get(url, headers=headers)
data = response.json()

In [52]:
data

{'headers': ['ROWHEADER', 'Test', 'ODI', 'T20', 'IPL'],
 'values': [{'values': ['Matches', '85', '204', '74', '254']},
  {'values': ['Innings', '128', '137', '41', '198']},
  {'values': ['Runs', '3886', '2806', '515', '3260']},
  {'values': ['Balls', '7037', '3284', '405', '2503']},
  {'values': ['Highest', '175', '87', '46', '77']},
  {'values': ['Average', '37.73', '32.63', '21.46', '27.86']},
  {'values': ['SR', '55.23', '85.45', '127.17', '130.25']},
  {'values': ['Not Out', '25', '51', '17', '81']},
  {'values': ['Fours', '383', '205', '39', '240']},
  {'values': ['Sixes', '75', '54', '14', '117']},
  {'values': ['Ducks', '8', '6', '3', '9']},
  {'values': ['50s', '27', '13', '0', '5']},
  {'values': ['100s', '5', '0', '0', '0']},
  {'values': ['200s', '0', '0', '0', '0']},
  {'values': ['300s', '0', '0', '0', '0']},
  {'values': ['400s', '0', '0', '0', '0']}],
 'appIndex': {'seoTitle': 'Ravindra Jadeja Profile - Cricbuzz | Cricbuzz.com',
  'webURL': 'http://www.cricbuzz.com/profi

In [53]:
url = "https://cricbuzz-cricket.p.rapidapi.com/stats/v1/player/587/bowling"

headers = {
	"x-rapidapi-key": "fe9c1d0ff9mshf8e519597a3627fp1b6d45jsn79f69a8379bc",
	"x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
}

response = requests.get(url, headers=headers)
data = response.json()

In [54]:
data

{'headers': ['ROWHEADER', 'Test', 'ODI', 'T20', 'IPL'],
 'values': [{'values': ['Matches', '85', '204', '74', '254']},
  {'values': ['Innings', '159', '196', '71', '225']},
  {'values': ['Balls', '19326', '10116', '1356', '4056']},
  {'values': ['Runs', '8305', '8180', '1612', '5188']},
  {'values': ['Maidens', '742', '59', '4', '2']},
  {'values': ['Wickets', '330', '231', '54', '170']},
  {'values': ['Avg', '25.17', '35.41', '29.85', '30.52']},
  {'values': ['Eco', '2.58', '4.85', '7.13', '7.67']},
  {'values': ['SR', '58.56', '43.79', '25.11', '23.86']},
  {'values': ['BBI', '7/42', '5/33', '3/15', '5/16']},
  {'values': ['BBM', '10/110', '5/33', '3/15', '5/16']},
  {'values': ['4w', '14', '7', '0', '3']},
  {'values': ['5w', '15', '2', '0', '1']},
  {'values': ['10w', '3', '0', '0', '0']}],
 'appIndex': {'seoTitle': 'Ravindra Jadeja Profile - Cricbuzz | Cricbuzz.com',
  'webURL': 'http://www.cricbuzz.com/profiles/587/ravindra-jadeja'},
 'seriesSpinner': [{'seriesName': 'Career'}]}

In [68]:

headers = {
    "x-rapidapi-key": "ac9352cb48msh05322e9d78564bfp133fa0jsn499d30d09b05",
    "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
}

players = [
    (8683, "Shardul Thakur"),
    (8808, "Axar Patel"),
    (9647, "Hardik Pandya"),
    (10945, "Washington Sundar"),
    (11195, "Shivam Dube"),
    (12086, "Abhishek Sharma"),
    (14701, "Nitish Kumar Reddy")
]

for player_id, name in players:
    batting_url = f"https://cricbuzz-cricket.p.rapidapi.com/stats/v1/player/{player_id}/batting"
    bowling_url = f"https://cricbuzz-cricket.p.rapidapi.com/stats/v1/player/{player_id}/bowling"

    # Fetch batting
    bat_res = requests.get(batting_url, headers=headers)
    bat_data = bat_res.json()

    runs = None
    for row in bat_data.get("values", []):
        if row["values"][0] == "Runs":
            runs = row["values"][1]  # ODI runs
            break

    # Fetch bowling
    bowl_res = requests.get(bowling_url, headers=headers)
    bowl_data = bowl_res.json()

    wickets = None
    for row in bowl_data.get("values", []):
        if row["values"][0] == "Wickets":
            wickets = row["values"][1]  # ODI wickets
            break

    # Update DB
    cursor.execute("""
        UPDATE ind_allrounder_players 
        SET runs = %s, wickets = %s 
        WHERE id = %s
    """, (runs, wickets, player_id))

    conn.commit()
    print(f"Updated {name} → Runs: {runs}, Wickets: {wickets}")

Updated Shardul Thakur → Runs: 377, Wickets: 33
Updated Axar Patel → Runs: 646, Wickets: 55
Updated Hardik Pandya → Runs: 532, Wickets: 17
Updated Washington Sundar → Runs: 752, Wickets: 32
Updated Shivam Dube → Runs: 0, Wickets: 0
Updated Abhishek Sharma → Runs: 0, Wickets: 0
Updated Nitish Kumar Reddy → Runs: 343, Wickets: 8
